## <center>Synchronous Computing: Heat Exchange</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 3620 </center>

<center> <img src="pictures/heatexchange1.png" width="700"/> 
</center>

<center> <img src="pictures/heatexchange2.png" width="700"/> 
</center>

<center> <img src="pictures/heatexchange3.png" width="700"/> 
</center>

<center> <img src="pictures/heatexchange4.png" width="700"/> 
</center>

<center> <img src="pictures/heatexchange5.png" width="700"/> 
</center>

#### <center> Row Exchange

- All processes interact with adjacent neighbors to exchange *ghost row* (*halo row*)
- All processes are to be synchronized such that the next iteration of the computation only happens after all row exchanges are completed

In [7]:
import ipyparallel
c=ipyparallel.Client(profile="mpicluster")
print(c.ids)

[0, 1, 2, 3, 4, 5, 6, 7]


In [8]:
%%px
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); N = 24
status = MPI.Status()

local_array = np.full(shape=(int(N / size) + 2, N), fill_value=rank, dtype="int")
full_array = np.zeros(shape=(N + size * 2, N), dtype="int")

comm.Gather(local_array, full_array, root=0)
if rank == 0:
    print (full_array)
if (rank < size - 1):
    print (rank)
    comm.Send(local_array[int(N/size),], dest = rank + 1, tag = 0)
    comm.Recv(local_array[int(N/size) + 1,], source = rank + 1, tag = 0, status = status)
if (rank > 0):
    print (rank)
    comm.Recv(local_array[0,], source = rank - 1, tag = 0, status = status)
    comm.Send(local_array[1,], dest = rank - 1, tag = 0)

comm.Barrier()
comm.Gather(local_array, full_array, root=0)
if rank == 0:
    print (full_array)

[stdout:0] 
4
4
[stdout:1] 
6
6
[stdout:2] 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 

In [9]:
import ipyparallel
c=ipyparallel.Client(profile="mpicluster")
print(c.ids)

[0, 1, 2, 3, 4, 5, 6, 7]


In [10]:
%%px
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); status = MPI.Status();

steps = 50
room_width = 1024
room_height = room_width

width = room_width + 2 # account for walls

# initialize room cells at 20 degrees. Each local slice has two additional rows and columns to account for 
# the ghost rows and the walls. 
local_slice = np.full(shape=(int(room_height / size) + 2, width), fill_value=20, dtype="float")
old_slice = np.zeros(shape=(int(room_height / size) + 2, width), dtype="float")
room_array = np.zeros(shape=(room_height, width), dtype="float")

if rank == 0:
    # initialize the fire place at the top of the room
    for i in range (int(width * 3 / 10 + 1), int(width * 7 / 10 + 1)):
        local_slice[0,i] = 300

# test room temperature print out:
comm.Gather(local_slice[1:int(room_height / size) + 1,], room_array, root=0)
#if rank == 0:
#    print (room_array)

#print(local_slice)

for step in range (0, steps):
    old_slice = local_slice
    if (rank < size - 1):
        comm.Send(old_slice[int(room_height/size),], dest = rank + 1, tag = 0)
        comm.Recv(old_slice[int(room_height/size) + 1,], source = rank + 1, tag = 0, status = status)
    if (rank > 0):
        comm.Recv(old_slice[0,], source = rank - 1, tag = 0, status = status)
        comm.Send(old_slice[1,], dest = rank - 1, tag = 0)
    comm.Barrier()
    
    for i in range (1,int(room_height/size) + 1):
        for j in range (1, room_width + 1):
            local_slice[i,j] = (old_slice[i - 1,j] + old_slice[i + 1,j] + old_slice[i, j - 1] + old_slice[i, j + 1])/4

comm.Gather(local_slice[1:int(room_height / size) + 1,], room_array, root=0)
if rank == 0:
    RED = "15 00 00 "
    ORANGE = "15 05 00 "
    YELLOW = "15 10 00 "
    LTGREEN = "00 13 00 "
    GREEN = "05 10 00 "
    LTBLUE = "00 05 10 "
    BLUE = "00 00 10 "
    DARKTEAL = "00 05 05 "
    BROWN = "03 03 00 "
    BLACK = "00 00 00 "

    colors = [RED, ORANGE, YELLOW, LTGREEN, GREEN, LTBLUE, BLUE, DARKTEAL, BROWN, BLACK]
    linelen = room_width;
    numlines = room_height;

    pnm_file = open("/home/lngo/git/cpsc-3620/heat.pnm", "w")
    pnm_file.write("P3\n%d %d\n15\n" % (linelen, numlines))

    for i in range(0,numlines):
        for j in range(0,linelen):
            if room_array[i,j] > 275:
                color = 0
            elif room_array[i,j] > 250:
                color = 1
            elif room_array[i,j] > 225:
                color = 2
            elif room_array[i,j] > 200:
                color = 3
            elif room_array[i,j] > 175:
                color = 4
            elif room_array[i,j] > 150:
                color = 5
            elif room_array[i,j] > 125:
                color = 6
            elif room_array[i,j] > 100:
                color = 7
            elif room_array[i,j] > 50:
                color = 8
            else:
                color = 9
            pnm_file.write("%s " % (colors[color]))
        pnm_file.write("\n")

    pnm_file.close()

In [5]:
!convert heat.pnm heat.gif

#### <center> The Game of Life </center>

- Board game: theoretically infinite two-dimensional array of cells
- Each cell hold one "organism" and has eight neighboring cells, including those diagonally adjacent
- Initially, some cells occupied

** Rules: **
- two or three neighboring organisms: survives for the next generation
- more than four neighbors: dies from overpopulation
- less than two neighbors: dies from isolation
- Empty cell with exactly three neighbors: spawn new organism

** Serious applications: **
- fluid/gas dynamics
- movement of fluids and gases aroudn objects
- diffusion of gases
- biological growth
- airflow across an airplane wing
- erosion/movement of sand at a beach or riverbank